In [1]:
import os
import csv
import numpy as np 
import pandas as pd
import re
import fnmatch

In [2]:
#Set working directory - can optionally be set somewhere else
os.chdir(os.getcwd())

In [3]:
#make a list of all the chrom_data files
file_list = []
pattern = '*.asc'
path = '..'
#os.chdir(data_folder)

#use os to crawl through all the files in the directory
for dirpath, dirnames, filenames in os.walk(path):

    if not filenames:
        continue
    
    ##makes a list of files that matches .asc 
    csv_files = fnmatch.filter(filenames, pattern)
    if csv_files:
        for file in csv_files:
            #print('{}/{}'.format(dirpath, file))
            file_list.append('{}/{}'.format(dirpath, file))

file_list = [c for c in file_list if "_data" in c]            

In [4]:
#This is the SM spreadsheet column headers
SM_column_names=[
    "Project Source Type (FB/UP/DN)",
    "Source (UP/DN) range",
    "Source Description",
    "Start volume (ml)",
    "pH Target",
    "Conductivity target (mS/cm)",
    "Buffer addition (identity, volumes, and resulting pH)",
    "Acid addition (identity, volumes, and resulting pH)",
    "Base addition (identity, volumes, and resulting pH)",
    "Conductivity adjustment (identity, volumes, and resulting pH)",
    "Filtration (Yes/No, filter identity)",
    "Notes",
    "Final Conductivity (mS/cm)",
    "Final pH",
    "Final Approximate Concentation (mg/ml)",
    "Final Total Volume (ml)"]
SM_column_names

['Project Source Type (FB/UP/DN)',
 'Source (UP/DN) range',
 'Source Description',
 'Start volume (ml)',
 'pH Target',
 'Conductivity target (mS/cm)',
 'Buffer addition (identity, volumes, and resulting pH)',
 'Acid addition (identity, volumes, and resulting pH)',
 'Base addition (identity, volumes, and resulting pH)',
 'Conductivity adjustment (identity, volumes, and resulting pH)',
 'Filtration (Yes/No, filter identity)',
 'Notes',
 'Final Conductivity (mS/cm)',
 'Final pH',
 'Final Approximate Concentation (mg/ml)',
 'Final Total Volume (ml)']

In [5]:
## 17 columns for SM spread sheet 
## these are the corresponeding key phrases in the AKTA/Unicorn notes
SM_phrases= [
     "Source Type (FB/UP/DN):", # 'Project Source Type (FB/UP/DN)',
     "Source (UP/DN) range:",#'Source (UP/DN) range',
     "Source Description:",#'Source Description',
    "Start volume (ml):", #'Start volume (ml)',
     "pH Target:", #'pH Target',
     "Conductivity target (mS/cm):",#'Conductivity target (mS/cm)',
     "Buffer addition:",#'Buffer addition (identity, volumes, and resulting pH)',
     "Acid addition:",#'Acid addition (identity, volumes, and resulting pH)',
     "Base addition:",#'Base addition (identity, volumes, and resulting pH)',
     "Conductivity adjustment:",#'Conductivity adjustment (identity, volumes, and resulting pH)',
     "Filtration:",#'Filtration (Yes/No, filter identity)',
     "Notes:",#'Notes',
     "Load conductivity (mS/cm):",#'Final Conductivity (mS/cm)',
     "Load pH:",#'Final pH',
     "Load A280 titer (mg/ml):",#'Final Approximate Concentation (mg/ml)',
     "Load total volume (ml):"]#'Final Total Volume (ml)']
SM_phrases

['Source Type (FB/UP/DN):',
 'Source (UP/DN) range:',
 'Source Description:',
 'Start volume (ml):',
 'pH Target:',
 'Conductivity target (mS/cm):',
 'Buffer addition:',
 'Acid addition:',
 'Base addition:',
 'Conductivity adjustment:',
 'Filtration:',
 'Notes:',
 'Load conductivity (mS/cm):',
 'Load pH:',
 'Load A280 titer (mg/ml):',
 'Load total volume (ml):']

In [6]:
print(len(SM_column_names))
print(len(SM_phrases))

16
16


In [7]:
#This is the DN spread sheet columns
DN_column_names= [
    "DN",
    "Project ID",
    "Study name",
    "Description of purpose",
    "Residence time",
    "Equilibration / wash variation",
    "Load variation",
    "Elution variation",
    "Load conductivity (mS/cm)",
    "Load pH",
    "Load A280 titer (mg/ml)",
    "Load total volume (ml)",
    "Method name",
    "Date of creation: ",
    "Created for system",
    "Result name",
    "Result folder name",
    "Flow rate",
    "Column ID",
    "SM ID"]
DN_column_names

['DN',
 'Project ID',
 'Study name',
 'Description of purpose',
 'Residence time',
 'Equilibration / wash variation',
 'Load variation',
 'Elution variation',
 'Load conductivity (mS/cm)',
 'Load pH',
 'Load A280 titer (mg/ml)',
 'Load total volume (ml)',
 'Method name',
 'Date of creation: ',
 'Created for system',
 'Result name',
 'Result folder name',
 'Flow rate',
 'Column ID',
 'SM ID']

In [8]:
## 22 columns for DN spread sheet 
## these are the corresponeding key phrases in the AKTA/Unicorn notes
DN_phrases= [
    "Result name: ", #'DN', 
    "Result folder name: ", #'Project ID',
    "Study name:", #'Study name',
    "Description of purpose:", #'Description of purpose',
    "Max flow rate	", #'Residence time',
    "Equilibration / wash variation:", #'Equilibration / wash variation',
    "Load variation:", #'Load variation',
    "Elution variation:",#'Elution variation',
    "Load conductivity (mS/cm):",#'Load conductivity (mS/cm)',
    "Load pH:", #'Load pH',
    "Load A280 titer (mg/ml):", #'Load A280 titer (mg/ml)',
    "Load total volume (ml):", #'Load total volume (ml)',
    "Method name: ", #'Method name',
    "Date of creation:", #'Date of creation: ',
    "Created for system:", #'Created for system',
    "Result name: ", #'Result name',
    "Result folder name: ",#'Result folder name',
    "Max flow rate	", #'Flow rate',
    "Column ID: ",#'Column ID',
    "Result name: "] #'SM ID'
DN_phrases

['Result name: ',
 'Result folder name: ',
 'Study name:',
 'Description of purpose:',
 'Max flow rate\t',
 'Equilibration / wash variation:',
 'Load variation:',
 'Elution variation:',
 'Load conductivity (mS/cm):',
 'Load pH:',
 'Load A280 titer (mg/ml):',
 'Load total volume (ml):',
 'Method name: ',
 'Date of creation:',
 'Created for system:',
 'Result name: ',
 'Result folder name: ',
 'Max flow rate\t',
 'Column ID: ',
 'Result name: ']

In [9]:
print(len(DN_column_names))
print(len(DN_phrases))

20
20


In [10]:
##initialize data frame using column names
SM_steps_performed = pd.DataFrame(columns = SM_column_names)

##For each file in folder
for file in file_list:
    print(file)

    ##initialize result list and open file
    result = []
    with open(file, 'r') as f:
        lines = f.readlines()
        ##look at each line, if the line contains the phrase, keep the line the result list.
        for line in lines:
            for phrase in SM_phrases:
                if phrase in line:
                    result.append(line)

    ##put captured result in dataframe and print
    df = pd.DataFrame(result+SM_phrases, columns = ["method_info"])
    df = df.replace(r'\n','', regex=True)

    ##filter out "Sample_ID (Type: Variable) without unique identifier"
    df=df[~df['method_info'].str.contains('without unique identifier', regex=False)]
    df=df[~df['method_info'].str.contains('(e.g resin screening)', regex=False)]
    df=df[~df['method_info'].str.contains('(e.g. 1st round bioreactor material, capture resin selection, @ 40 mg/ml load, Fry)', regex=False)]
    df=df[~df['method_info'].str.contains('(buffer identity, pH, CV)', regex=False)]
    df=df[~df['method_info'].str.contains('(resin loading mg/ml, solvent, pH)', regex=False)]
    df=df[~df['method_info'].str.contains('(buffer identity, CV)', regex=False)]
    df=df[~df['method_info'].str.contains('(BRX pool)', regex=False)]
    df=df[~df['method_info'].str.contains('(buffer identity, CV)', regex=False)]
    df=df[~df['method_info'].str.contains('(identity, volumes, and resulting pH)', regex=False)]
    df=df[~df['method_info'].str.contains('(Yes/No, filter identity)', regex=False)]

    ##add string lengths and sort by string length 
    df['str_len']=df["method_info"].str.len()
    df = df.sort_values(by='str_len', ascending=False)
    print(df.to_string())
    
    ##filter the captured result for the longest answer for each phrase and fit into dataframe
    result = []
    for phrase in SM_phrases:
        entry = df[df["method_info"].str.contains(phrase, regex=False)]['method_info'].iloc[0]
        entry = entry.replace(phrase,'').strip()
        if entry == '': entry = 'NA'
        result.append(entry)   
    
    ##append the result from this file to main dataframe
    SM_steps_performed = pd.concat([SM_steps_performed,pd.DataFrame([result], columns = SM_column_names)])
    



In [11]:
##define function for checking string to float
def is_float_try_except(value):
    try: 
        float(value)
        return True
    except ValueError:
        return False

##initialize data frame using column names
DN_steps_performed = pd.DataFrame(columns = DN_column_names)

##For each file in folder
for file in file_list:
    print(file)

    ##initialize result list and open file
    result = []
    with open(file, 'r') as f:
        lines = f.readlines()
        ##look at each line, if the line contains the phrase, keep the line the result list.
        for line in lines:
            for phrase in DN_phrases:
                if phrase in line:
                    result.append(line)

    ##put captured result in dataframe and print
    df = pd.DataFrame(result+DN_phrases, columns = ["method_info"])
    df = df.replace(r'\n','', regex=True)

    ##filter out "Sample_ID (Type: Variable) without unique identifier"
    df=df[~df['method_info'].str.contains('without unique identifier', regex=False)]
    df=df[~df['method_info'].str.contains('(e.g resin screening)', regex=False)]
    df=df[~df['method_info'].str.contains('(e.g. 1st round bioreactor material, capture resin selection, @ 40 mg/ml load, Fry)', regex=False)]
    df=df[~df['method_info'].str.contains('(buffer identity, pH, CV)', regex=False)]
    df=df[~df['method_info'].str.contains('(resin loading mg/ml, solvent, pH)', regex=False)]
    df=df[~df['method_info'].str.contains('(buffer identity, CV)', regex=False)]
    df=df[~df['method_info'].str.contains('(BRX pool)', regex=False)]
    df=df[~df['method_info'].str.contains('(buffer identity, CV)', regex=False)]
    df=df[~df['method_info'].str.contains('(identity, volumes, and resulting pH)', regex=False)]
    df=df[~df['method_info'].str.contains('(Yes/No, filter identity)', regex=False)]

    ##add string lengths and sort by string length 
    df['str_len']=df["method_info"].str.len()
    df = df.sort_values(by='str_len', ascending=False)
    print(df.to_string())
    
    ##filter the captured result for the longest answer for each phrase and fit into dataframe
    result = []
    for phrase in DN_phrases:
        entry = df[df["method_info"].str.contains(phrase, regex=False)]['method_info'].iloc[0]
        entry = entry.replace(phrase,'').strip()
        if entry == '': entry = 'NA'
        result.append(entry)
    
    ##extract the DN #
    match = re.search(r"[A-Za-z]{2}\s?\d{3,4}",result[0])
    if match:
        result[0]=(match.group()[2:])
        result[19]=(match.group()[2:])
    else:
        result[0]=''
        result[19]=''

    ##extract the SI #
    match = re.search(r"[A-Za-z]{2}-\d{2,3}[A-Za-z]{1}\d{1,2}",result[1])
    if match:
        result[1]=(match.group())
    else:
        result[1]=''
    
    ##calculate residence time
    if is_float_try_except(result[4][:4]) and is_float_try_except(result[18][:4]):
        result[4]=round(float(result[18][:4])/float(result[4][:4]),2)
    else: result[4]='NA'
    
    ##append the result from this file to main dataframe
    DN_steps_performed = pd.concat([DN_steps_performed,pd.DataFrame([result], columns = DN_column_names)])

In [12]:
pd.concat([DN_steps_performed, SM_steps_performed], axis=1).to_csv("SMDN_Unicorn_steps.csv")

In [13]:
SM_steps_performed

,Project Source Type (FB/UP/DN),Source (UP/DN) range,Source Description,Start volume (ml),pH Target,Conductivity target (mS/cm),"Buffer addition (identity, volumes, and resulting pH)","Acid addition (identity, volumes, and resulting pH)","Base addition (identity, volumes, and resulting pH)","Conductivity adjustment (identity, volumes, and resulting pH)","Filtration (Yes/No, filter identity)",Notes,Final Conductivity (mS/cm),Final pH,Final Approximate Concentation (mg/ml),Final Total Volume (ml)
